In [10]:
%load_ext autoreload
%autoreload 2

import torch
import pandas as pd

from seqfacben.generators.random import RandomSequenceGenerator
from seqfacben.tasks.sorting import SortingTask
from seqfacben.tasks.copy import CopyTask
from seqfacben.models.simple_nn import SimpleNN
from seqfacben.manager.task_manager import TaskManager


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### I. Single Combination

In [23]:
# Parameters
device = "cuda" if torch.cuda.is_available() else "cpu"

# Generator parameters
vocab_size = 200
seq_len = 10

# Training parameters
batch_size = 64
train_steps = 10000

# Training loop
results = []

# Set up and train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator = RandomSequenceGenerator(seq_len, vocab_size)
task = CopyTask(generator)
task = SortingTask(generator)
model = SimpleNN(vocab_size, seq_len, d_model=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

manager = TaskManager(task, model, optimizer, device)

# Train
manager.train(n_steps=10000, batch_size=32)

# Test
print("\nTesting:")
acc = manager.eval_step(batch_size=32)
print(f"Final accuracy: {acc:.4f}")

manager.show_examples(n_examples=3)

Step 100: loss = 4.5627, acc = 0.0000
Step 200: loss = 4.4342, acc = 0.0000
Step 300: loss = 4.4245, acc = 0.0000
Step 400: loss = 4.0265, acc = 0.0000
Step 500: loss = 3.9754, acc = 0.0000
Step 600: loss = 3.5508, acc = 0.0000
Step 700: loss = 3.4714, acc = 0.0000
Step 800: loss = 3.1303, acc = 0.0000
Step 900: loss = 2.9841, acc = 0.0000
Step 1000: loss = 2.8882, acc = 0.0000
Step 1100: loss = 2.6664, acc = 0.0000
Step 1200: loss = 2.7652, acc = 0.0000
Step 1300: loss = 2.7276, acc = 0.0000
Step 1400: loss = 2.5291, acc = 0.0000
Step 1500: loss = 2.4649, acc = 0.0000
Step 1600: loss = 2.3657, acc = 0.0000
Step 1700: loss = 2.2465, acc = 0.0000
Step 1800: loss = 2.2839, acc = 0.0000
Step 1900: loss = 2.2980, acc = 0.0000
Step 2000: loss = 2.3256, acc = 0.0000
Step 2100: loss = 2.0513, acc = 0.0000
Step 2200: loss = 1.9953, acc = 0.0000
Step 2300: loss = 1.9070, acc = 0.0000
Step 2400: loss = 1.9256, acc = 0.0000
Step 2500: loss = 2.0288, acc = 0.0000
Step 2600: loss = 1.8615, acc = 0.

##### II. Multi Experiment Example

In [22]:
import pandas as pd

# Experiment configurations
configs = [
    {'vocab_size': 5, 'seq_len': 10},
    {'vocab_size': 10, 'seq_len': 10},
    {'vocab_size': 20, 'seq_len': 10},
    {'vocab_size': 5, 'seq_len': 20},
    {'vocab_size': 5, 'seq_len': 30},
]

results = []

for config in configs:
    vocab_size = config['vocab_size']
    seq_len = config['seq_len']
    
    print(f"\n{'='*60}")
    print(f"Running: vocab_size={vocab_size}, seq_len={seq_len}")
    print(f"{'='*60}")
    
    # Setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    generator = RandomSequenceGenerator(seq_len, vocab_size)
    task = SortingTask(generator)
    model = SimpleNN(vocab_size, seq_len, d_model=64).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    manager = TaskManager(task, model, optimizer, device)
    
    # Train
    history = manager.train(n_steps=10000, batch_size=32, eval_every=500)
    
    # Final evaluation
    final_acc = manager.eval_step(batch_size=256)
    
    # Save results
    results.append({
        'vocab_size': vocab_size,
        'seq_len': seq_len,
        'final_accuracy': final_acc,
        'final_loss': history[-1]['loss'] if history else None
    })
    
    print(f"\nFinal accuracy: {final_acc:.4f}")

# Create DataFrame
df = pd.DataFrame(results)
print("\n" + "="*60)
print("RESULTS SUMMARY")
print("="*60)
print(df)
df.to_csv('sorting_experiments.csv', index=False)


Running: vocab_size=5, seq_len=10
Step 500: loss = 0.0130, acc = 0.9688
Step 1000: loss = 0.0022, acc = 0.9688
Step 1500: loss = 0.0034, acc = 1.0000
Step 2000: loss = 0.0005, acc = 1.0000
Step 2500: loss = 0.0009, acc = 1.0000
Step 3000: loss = 0.0501, acc = 1.0000
Step 3500: loss = 0.0013, acc = 1.0000
Step 4000: loss = 0.0000, acc = 0.9688
Step 4500: loss = 0.0022, acc = 1.0000
Step 5000: loss = 0.0001, acc = 1.0000
Step 5500: loss = 0.0001, acc = 1.0000
Step 6000: loss = 0.0037, acc = 0.9688
Step 6500: loss = 0.0003, acc = 1.0000
Step 7000: loss = 0.0005, acc = 1.0000
Step 7500: loss = 0.0001, acc = 1.0000
Step 8000: loss = 0.0003, acc = 0.9688
Step 8500: loss = 0.0479, acc = 0.9688
Step 9000: loss = 0.0213, acc = 0.9688
Step 9500: loss = 0.0796, acc = 0.9375
Step 10000: loss = 0.0129, acc = 0.9688

Final accuracy: 0.9922

Running: vocab_size=10, seq_len=10
Step 500: loss = 0.1542, acc = 0.9688
Step 1000: loss = 0.0235, acc = 0.9688
Step 1500: loss = 0.0161, acc = 0.9062
Step 2000